In [ ]:
# Imports and Configuration
!pip install tensorflow --quiet
import os
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timezone, timedelta
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import sys
import warnings
import gradio as gr

warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

INPUT_STEPS = 8
RESAMPLE_RULE = '15T'
MIN_TRAINING_SAMPLES = 15
CSV_FILE = 'carpark_multi_snapshots_class.csv'
MODEL_DIR = 'best_lstm_classifiers'
EPOCHS = 30
BATCH_SIZE = 32

# API Config
USERNAME = "solobalama"
PASSWORD = "coder#346"
STATIC_URL = "https://www.netraveldata.co.uk/api/v2/carpark/static"
DYNAMIC_URL = "https://www.netraveldata.co.uk/api/v2/carpark/dynamic"
HEADERS = {"User-Agent": "parking-ml-demo/lstm-classifier-v1"}

STATE_MAPPING = {
    'SPACES': 0, 'OPEN': 0, 'ALMOST FULL': 1, 'FULL': 2, 'CLOSED': 3,
    'UNKNOWN': 4, 'FAULTY': 5
}
STATE_DECODER = {v: k for v, k in STATE_MAPPING.items()}
STATE_ENCODER = LabelEncoder()
STATE_ENCODER.fit(list(STATE_MAPPING.keys()))

LabelEncoder()

In [ ]:
# Data Fetching and Preparation Functions
def get_records(data):
    if isinstance(data, list):
        return data
    elif isinstance(data, dict):
        for k in ('assets', 'data', 'carparks', 'items'):
            if k in data and isinstance(data[k], list):
                return data[k]
    return []

def fetch_static(username, password):
    try:
        r = requests.get(STATIC_URL, auth=(username, password), headers=HEADERS, timeout=20)
        r.raise_for_status()
        records = get_records(r.json())
        rows = []
        for a in records:
            try:
                scode = a.get('systemCodeNumber') or a.get('systemCode')
                defs = a.get('definitions') or a
                if isinstance(defs, list) and defs:
                    defs = defs[0]
                short = defs.get('shortDescription') if isinstance(defs, dict) else None
                conf = a.get('configurations') or a.get('configuration')
                capacity = None
                if isinstance(conf, list) and conf:
                    capacity = conf[0].get('capacity') or conf[0].get('carParkCapacity')
                elif isinstance(conf, dict):
                    capacity = conf.get('capacity') or conf.get('carParkCapacity')
                rows.append({
                    'systemCodeNumber': scode,
                    'shortDescription': short,
                    'capacity': capacity
                })
            except:
                continue
        return pd.DataFrame(rows).dropna(subset=['systemCodeNumber']).drop_duplicates('systemCodeNumber')
    except Exception as e:
        print(f"Error fetching static data: {e}", file=sys.stderr)
        return pd.DataFrame()

def fetch_dynamic(username, password):
    try:
        r = requests.get(DYNAMIC_URL, auth=(username, password), headers=HEADERS, timeout=20)
        r.raise_for_status()
        records = get_records(r.json())
        rows = []
        for a in records:
            try:
                scode = a.get('systemCodeNumber') or a.get('systemCode')
                dyn = a.get('dynamics') or a
                if isinstance(dyn, list) and dyn:
                    dyn = dyn[0]
                occ = dyn.get('occupancy') if isinstance(dyn, dict) else None
                state = dyn.get('occupancyStateDescription') or dyn.get('stateDescription')
                rows.append({
                    'systemCodeNumber': scode,
                    'occupancy': occ,
                    'stateDescription': state
                })
            except:
                continue
        return pd.DataFrame(rows).dropna(subset=['systemCodeNumber'])
    except Exception as e:
        print(f"Error fetching dynamic data: {e}", file=sys.stderr)
        return pd.DataFrame()

def fetch_and_save(username, password, out_csv=CSV_FILE):
    ds = fetch_static(username, password)
    dd = fetch_dynamic(username, password)

    merged = pd.merge(ds, dd, on='systemCodeNumber', how='left')
    merged['fetched_at'] = pd.to_datetime(datetime.now(timezone.utc))

    merged['occupancy'] = pd.to_numeric(merged['occupancy'], errors='coerce')
    merged['capacity'] = pd.to_numeric(merged['capacity'], errors='coerce')

    merged['availableSpaces'] = merged['capacity'] - merged['occupancy']
    merged['availableSpaces'] = merged['availableSpaces'].clip(lower=0)

    merged['state_encoded'] = merged['stateDescription'].str.upper().map(
        STATE_MAPPING
    ).fillna(STATE_MAPPING['UNKNOWN'])

    merged_for_save = merged[[
        'systemCodeNumber', 'shortDescription', 'capacity',
        'availableSpaces', 'state_encoded', 'fetched_at'
    ]].dropna(subset=['capacity', 'state_encoded'])

    file_exists = os.path.exists(out_csv)
    merged_for_save.to_csv(out_csv, mode='a', header=(not file_exists), index=False)

    return merged_for_save.drop_duplicates(subset=['systemCodeNumber'], keep='last')


def build_timeseries(csv_path=CSV_FILE):
    df_snap = pd.read_csv(csv_path, parse_dates=['fetched_at'])
    parks_data = {}

    for pid, grp in df_snap.groupby('systemCodeNumber'):
        latest_info = grp.drop_duplicates(
            subset=['systemCodeNumber'], keep='last'
        ).iloc[0]
        cap = float(latest_info['capacity'])
        name = latest_info['shortDescription']

        state_series = grp.set_index('fetched_at')['state_encoded'].resample(
            RESAMPLE_RULE
        ).last().ffill().bfill().astype(int)

        parks_data[pid] = {
            'class_series': state_series,
            'capacity': cap,
            'name': name
        }
    return parks_data